In [1]:
# TensorFlow and keras
import tensorflow as tf
from tensorflow import keras
import keras.backend as K
from keras.models import Sequential
# Helper libraries
import os
from glob import glob
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import logging
logger = tf.get_logger()
logger.setLevel(logging.ERROR)
import cv2
from keras.models import Model
from keras.layers import add
from keras.layers import Activation
from keras.layers import Input
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import AveragePooling2D
from keras.layers import GlobalAveragePooling2D
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers.merge import concatenate
from keras.utils import plot_model
from albumentations import (
    Compose, HorizontalFlip, CLAHE, HueSaturationValue,
    RandomBrightness, RandomContrast, RandomGamma,
    ToFloat, ShiftScaleRotate
)

Using TensorFlow backend.


In [0]:
# function for creating a vgg block
def vgg_block(layer_in, n_filters, n_conv):
	# add convolutional layers
	for _ in range(n_conv):
		layer_in = Conv2D(n_filters, (3,3), padding='same', activation='relu')(layer_in)
	# add max pooling layer
	layer_in = MaxPooling2D((2,2), strides=(2,2))(layer_in)
	return layer_in

In [0]:
# function for creating an identity or projection residual module
def residual_module(layer_in, n_filters):
	merge_input = layer_in
	# check if the number of filters needs to be increase, assumes channels last format
	if layer_in.shape[-1] != n_filters:
		merge_input = Conv2D(n_filters, (1,1), padding='same', activation='relu', kernel_initializer='he_normal')(layer_in)
	# conv1
	conv1 = Conv2D(n_filters, (3,3), padding='same', activation='relu', kernel_initializer='he_normal')(layer_in)
	# conv2
	conv2 = Conv2D(n_filters, (3,3), padding='same', activation='linear', kernel_initializer='he_normal')(conv1)
	# add filters, assumes filters/channels last
	layer_out = add([conv2, merge_input])
	# activation function
	layer_out = Activation('relu')(layer_out)
	return layer_out

In [0]:
# function for creating a naive inception block
def inception_module(layer_in, f1, f2, f3):
	# 1x1 conv
	conv1 = Conv2D(f1, (1,1), padding='same', activation='relu')(layer_in)
	# 3x3 conv
	conv3 = Conv2D(f2, (3,3), padding='same', activation='relu')(layer_in)
	# 5x5 conv
	conv5 = Conv2D(f3, (5,5), padding='same', activation='relu')(layer_in)
	# 3x3 max pooling
	pool = MaxPooling2D((3,3), strides=(1,1), padding='same')(layer_in)
	# concatenate filters, assumes filters/channels last
	layer_out = concatenate([conv1, conv3, conv5, pool], axis=-1)
	return layer_out

In [0]:
def build_model_VGG():

  inputShape = (256, 256,3)
  inputs = Input(shape = inputShape)
  x = vgg_block(inputs,3,2)
  x = AveragePooling2D((7, 7))(x)
  x = Flatten()(x)
  x = Dense(2)(x)
  x = Activation("softmax")(x)
  # create the model
  model = Model(inputs, x, name="model")
	# return the constructed network architecture
  return model

In [0]:
def build_model_ResNet():

  inputShape = (256, 256,3)
  inputs = Input(shape = inputShape)
  x = residual_module(inputs,3)
  x = AveragePooling2D((7, 7))(x)
  x = Flatten()(x)
  x = Dense(2)(x)
  x = Activation("softmax")(x)
  # create the model
  model = Model(inputs, x, name="model")
	# return the constructed network architecture
  return model

In [0]:
def build_model_inception():

  inputShape = (256, 256,3)
  inputs = Input(shape = inputShape)
  x = inception_module(inputs,64, 128, 32)
  x = AveragePooling2D((7, 7))(x)
  x = Flatten()(x)
  x = Dense(2)(x)
  x = Activation("softmax")(x)
  # create the model
  model = Model(inputs, x, name="model")
	# return the constructed network architecture
  return model